In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [2]:
diamonds_df = pd.read_csv('../data/diamonds_train.csv')

In [3]:
diamonds_predict = pd.read_csv('../data/diamonds_test.csv')

In [15]:
diamonds_df['volume'] = (diamonds_df['x']*diamonds_df['y']*diamonds_df['z']).round(2)
diamonds_predict['volume'] = (diamonds_predict['x']*diamonds_predict['y']*diamonds_predict['z']).round(2)

diamonds_df['density'] = (diamonds_df['carat']/diamonds_df['volume']).round(5)
diamonds_predict['density'] = (diamonds_predict['carat']/diamonds_predict['volume']).round(5)

In [16]:
diamonds_df

,carat,cut,color,clarity,depth,table,price,x,y,z,volume,density
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25,197.10,0.00614
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75,52.40,0.00611
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65,113.44,0.00626
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00,66.27,0.00619
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95,168.43,0.00606
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,Ideal,G,VS1,62.7,57.0,10070,7.10,7.04,4.43,221.43,0.00605
40451,2.02,Good,F,SI2,57.1,60.0,12615,8.31,8.25,4.73,324.28,0.00623
40452,1.01,Ideal,H,SI1,62.7,56.0,5457,6.37,6.42,4.01,163.99,0.00616
40453,0.33,Ideal,J,VS1,61.9,54.3,456,4.45,4.47,2.76,54.90,0.00601


In [29]:
NUM_FEATS = ['carat','table','depth','x','y','z','volume']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS
TARGET = 'price'

In [30]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                ('scaler', StandardScaler())])

In [31]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [32]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                                ('cat', categorical_transformer, CAT_FEATS)])

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
diamonds_train, diamonds_test = train_test_split(diamonds_df)

In [35]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 12)
(10114, 12)


In [36]:

from lightgbm import LGBMRegressor

model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor',LGBMRegressor())])

In [37]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

In [38]:
from sklearn.metrics import mean_squared_error

In [39]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])

In [40]:
print(f"test error: {mean_squared_error(y_pred=y_test, y_true=diamonds_test[TARGET], squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=diamonds_train[TARGET], squared=False)}")

test error: 540.6499844636703
train error: 476.4509421644861


In [42]:
from sklearn.model_selection import cross_val_score

In [44]:
scores = cross_val_score(model,  
                         diamonds_df[FEATS], 
                         diamonds_df[TARGET], 
                         scoring='neg_root_mean_squared_error', 
                         cv=10, n_jobs=-1)

In [45]:
import numpy as np
np.mean(-scores)

545.0907347220104

In [46]:
from sklearn.model_selection import RandomizedSearchCV

In [48]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds_df[FEATS], diamonds_df[TARGET])


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5; 1/32] START preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128
[CV 1/5; 1/32] END preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128; total time=   0.9s
[CV 2/5; 1/32] START preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128
[CV 2/5; 1/32] END preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128; total time=   0.6s
[CV 3/5; 1/32] START preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128
[CV 3/5; 1/32] END preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128; total time=   0.6s
[CV 4/5; 1/32] START preprocessor__num__imputer__strategy=median, regressor__max_depth=4, regressor__n_estimators=128
[CV 4/5; 1/32] END preprocessor__num__imputer__strategy=me

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'table',
                                                                                'depth',
                                                                                'x',
              

In [49]:
grid_search.best_params_

{'regressor__n_estimators': 256,
 'regressor__max_depth': 8,
 'preprocessor__num__imputer__strategy': 'median'}

In [50]:
grid_search.best_score_

-542.9860238177051

In [51]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [52]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'], 'price': y_pred})

In [53]:
submission_df.to_csv('../data/resultados6.csv', index=False)